# 📘 Week 4 Day 4: 文本与序列 (Strings & NLP Prep)

## 1 核心概念：如何让计算机“读懂”字？

**底层联系 (The Connection):**

  * **LeetCode**: 字符串（String）是一串字符的数组。我们需要处理字符的顺序、查找子串等。
  * **AI (NLP)**: 神经网络不认识 "Apple"。
    1.  **Tokenization (分词)**: 把 "Apple is red" 切成 `["Apple", "is", "red"]`。
    2.  **Indexing (索引)**: 查字典，变成数字 `[102, 5, 99]`。
    3.  **Embedding**: 变成向量（Stage 2 重点）。

今天我们先解决前两步：**怎么优雅地处理字符串**。


## 2 AI 工程预热 (The "Job" Skills)

在进入算法题之前，我们先写一段代码，模拟 BERT 模型是如何处理输入的。这比直接做题更有“具身感”。

### 2.1 手写一个简易分词器 (Tokenizer)

运行以下代码，理解文本是如何变成 Tensor 的。

In [2]:
import torch

class SimpleTokenizer:
    def __init__(self, text_corpus):
        # 1. 构建词表 (Vocabulary)
        # set 去重 -> sorted 排序 -> enumerate 建立索引
        tokens = sorted(list(set(text_corpus.split())))
        self.token_to_id = {t: i+1 for i, t in enumerate(tokens)} # +1 是因为 0 通常留给 padding
        self.token_to_id['<UNK>'] = 0 # 未知词
        
        # 反向查找表 (ID -> Token)
        self.id_to_token = {v: k for k, v in self.token_to_id.items()}
        
    def encode(self, text):
        # 分词并转 ID
        words = text.split()
        ids = [self.token_to_id.get(w, 0) for w in words]
        return torch.tensor(ids)
    
    def decode(self, ids):
        # ID 转回文本
        if isinstance(ids, torch.Tensor):
            ids = ids.tolist()
        return " ".join([self.id_to_token.get(i, '<UNK>') for i in ids])

# --- 交互式测试 ---
# 1. 训练一个简单的词表
corpus = "i love ai and python machine learning is great"
tokenizer = SimpleTokenizer(corpus)
print(f"词表大小: {len(tokenizer.token_to_id)}")
print(f"词表内容: {tokenizer.token_to_id}")

# 2. 手动输入测试
print("\n=== 测试分词器 ===")
input_text = input("请输入一段英文 (例如: i love python): ").strip()

# 编码
encoded_ids = tokenizer.encode(input_text)
print(f"Token IDs (Tensor): {encoded_ids}")

# 解码
decoded_text = tokenizer.decode(encoded_ids)
print(f"还原文本: {decoded_text}")

词表大小: 10
词表内容: {'ai': 1, 'and': 2, 'great': 3, 'i': 4, 'is': 5, 'learning': 6, 'love': 7, 'machine': 8, 'python': 9, '<UNK>': 0}

=== 测试分词器 ===
Token IDs (Tensor): tensor([4, 7, 9])
还原文本: i love python


> **注意**: 如果你输入的单词不在 `corpus` 里（比如 "robot"），它会被转成 `0` (`<UNK>`)。这就是 NLP 中常见的 **OOV (Out Of Vocabulary)** 问题。

## 3 LeetCode 算法突击 (The "Interview" Skills)

字符串题目最考验对 **Edge Case (边界条件)** 的处理，比如空字符串、只有一个字符、全是空格等。

### 3.1 最长公共前缀 (Longest Common Prefix) - Easy

  * **LeetCode 链接**: [14. Longest Common Prefix](../../LeetCode%20practice/1-50.ipynb)
  * **AI 关联**: 在处理序列数据时，提取公共特征（Pattern Recognition）的基础逻辑。
  * **核心思路**: **横向扫描**或**排序后比较**。
      * 技巧：如果把字符串数组排序，只需要比较**第一个**和**最后一个**字符串即可（因为中间的肯定更像）。

In [ ]:
from typing import List

class Solution:
    def longestCommonPrefix(self, strs: List[str]) -> str:
        if not strs:
            return ""
        
        # 技巧：按字母序排序
        # 比如 ["flower", "flow", "flight"] -> ["flight", "flow", "flower"]
        # 只要比较 "flight" 和 "flower" 的公共部分，那就是所有人的公共部分
        strs.sort()
        
        first = strs[0]
        last = strs[-1]
        
        i = 0
        while i < len(first) and i < len(last) and first[i] == last[i]:
            i += 1
            
        return first[:i]

### 3.2 字符串相加 (Add Strings) - Easy (高频)

  * **LeetCode 链接**: [415. Add Strings](https://leetcode.com/problems/add-strings/)
  * **核心思路**: **模拟大数运算**。不能直接转 `int()`（题目通常限制），要一位一位算。双指针从后往前走，处理 `carry` (进位)。

In [ ]:
class Solution:
    def addStrings(self, num1: str, num2: str) -> str:
        i, j = len(num1) - 1, len(num2) - 1
        carry = 0
        res = []
        
        while i >= 0 or j >= 0 or carry:
            # 字符转数字: ord('5') - ord('0') = 5
            n1 = int(num1[i]) if i >= 0 else 0
            n2 = int(num2[j]) if j >= 0 else 0
            
            temp_sum = n1 + n2 + carry
            carry = temp_sum // 10
            res.append(str(temp_sum % 10))
            
            i -= 1
            j -= 1
            
        # 结果是反的，要倒回来
        return "".join(res[::-1])

# --- 交互式测试 ---
solution = Solution()
print("\n=== 测试字符串相加 ===")
n1 = input("请输入第一个数字 (字符串): ").strip()
n2 = input("请输入第二个数字 (字符串): ").strip()

result = solution.addStrings(n1, n2)
print(f"计算结果: {result}")

### 3.3 无重复字符的最长子串 (Longest Substring Without Repeating Characters) - Medium (经典必考)

  * **LeetCode 链接**: [3. Longest Substring Without Repeating Characters](../../LeetCode%20practice/1-50.ipynb)
  * **AI 关联**: **Attention Mechanism (注意力机制)** 的雏形。模型在看序列时，关注的是“Unique”的信息。
  * **核心思路**: **滑动窗口 + 哈希表**。
      * 右指针 `right` 主动扩张。
      * 一旦遇到重复字符，左指针 `left` 被动收缩（跳到重复字符的下一位）。

In [ ]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        dic, res, i = {}, 0, -1
        for j in range(len(s)):
            if s[j] in dic:
                i = max(dic[s[j]], i) # 更新左指针 i
            dic[s[j]] = j # 哈希表记录
            res = max(res, j - i) # 更新结果
        return res

## 4 总结与作业

### 今日复盘

1.  **NLP 基础**: 字符串 -\> Tokenizer -\> IDs -\> Tensor。这就是所有大模型（包括我 Gemini）理解你说话的第一步。
2.  **算法技巧**:
      * **双指针从后往前**：处理“进位”类问题的通用解法。
      * **Map + 滑动窗口**：解决“不重复”、“连续子串”类问题的核武器。

### 课后作业

1.  运行并手动测试以上代码。建议在测试“无重复子串”时，输入 `"pwwkew"`，看看结果是不是 3 (`wke`)，并理解为什么左指针会跳跃。
2.  **思考**: `SimpleTokenizer` 里我们用的是空格分词 (`split()`)。中文没有空格（比如“我爱人工智能”），该怎么办？（这将在 Stage 2 的 BERT 学习中揭晓）。